### Imports

In [ ]:
from breeze_connect import BreezeConnect
from jproperties import Properties
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

### Login to Breeze API

In [ ]:
configs = Properties()
with open('app.properties', 'rb') as read_prop:
    configs.load(read_prop,"utf-8")
if len(configs) == 0:
    print("Error while reading properties files")

api_key = configs.get("APP_KEY").data if configs.get("APP_KEY") else None
api_secret = configs.get("SECRET_KEY").data if configs.get("SECRET_KEY") else None
session_token =configs.get("SESSION_TOKEN").data if configs.get("SESSION_TOKEN") else None
breeze = BreezeConnect(api_key=api_key)
breeze.generate_session(api_secret=api_secret,session_token=session_token)

In [ ]:
# Generate ISO8601 Date/DateTime String
# iso_date_string = datetime.datetime.strptime("28/02/2021","%d/%m/%Y").isoformat()[:10] + 'T05:30:00.000Z'
# iso_date_time_string = datetime.datetime.strptime("28/02/2021 23:59:59","%d/%m/%Y %H:%M:%S").isoformat()[:19] + '.000Z'
current_date = datetime.now().strftime('%Y-%m-%dT%H:%M:%S.%f%z')
print(current_date)

In [ ]:
getDF = lambda data : pd.DataFrame(data["Success"])

In [ ]:
interval = "30minute" # "1minute", "5minute", "30minute" or as "1day"
current_date = "17-Apr-2023"
stock_code = "NIFTY"
exchange_code = "NSE"
product_type = "cash"

columnsToDrop = ['stock_code', 'product_type', 'exchange_code','expiry_date','right','strike_price','volume','open_interest',]

# get historical data for option premium for a particular strike price and underlying data
df = getDF(breeze.get_historical_data(interval,current_date,current_date,stock_code,exchange_code,product_type))
df =df.drop(columns =columnsToDrop)
df

In [ ]:
import numpy as np
stock_code = "NIFTY"
exchange_code = "NFO"
product_type = "options"
expiry_date = "27-Apr-2023"
right="call"

columnsToDrop = ['stock_code', 'product_type', 'exchange_code']
t=breeze.get_option_chain_quotes(stock_code,exchange_code,product_type=product_type,expiry_date=expiry_date,right=right)
df = getDF(t)
df =df.drop(columns =columnsToDrop)
df=df[df['ltp'] > 0] 
df['spot_price'] = df['spot_price'].astype('float')
# df['intrinsic_value'] = np.maximum(0,df['spot_price'] - df['strike_price'])
# df=df[df['intrinsic_value'] > 0] 
df=df[df['strike_price'] > 17600] 
df['intrinsic_value'] =df['spot_price'] - df['strike_price']
df['time_value'] = df['ltp'] - df['intrinsic_value']
df[['spot_price','strike_price','ltp','intrinsic_value','time_value']]

In [ ]:

df['spot_price']-df['strike_price']

In [ ]:
# bar plot
df.plot(kind='line',
        x='ltp',
        y='spot_price',
        color='green')
 

In [ ]:
t =breeze.get_option_chain_quotes(stock_code="ITC",
                    exchange_code="NFO",
                    product_type="options",
                    strike_price="400",
                    right="call")
printDF(t)

In [ ]:
printDF(breeze.get_portfolio_holdings(exchange_code="NSE",
                                from_date="2022-08-01T06:00:00.000Z",
                                to_date="2022-08-19T06:00:00.000Z",
                                stock_code="",
                                portfolio_type=""))